In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('display.max_columns', 999)

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow_core.estimator import inputs
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
def normalize_and_encode(dataframe):
    column_names_to_not_normalize = ['result']
    column_names_to_normalize = [x for x in list(dataframe) if x not in column_names_to_not_normalize ]
    x = dataframe[column_names_to_normalize].values
    x_scaled = preprocessing.normalize(x)
    df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = dataframe.index)
    dataframe[column_names_to_normalize] = df_temp

    le = preprocessing.LabelEncoder()
    le.fit([ "H", "A", "D"])
    dataframe.loc[:,['result']]=le.transform(dataframe['result'])
    
    return dataframe

def get_X_and_y(dataframe):
    X = dataframe.drop(columns=['result']).values
    y = dataframe[['result']].values
    return X,y

df01 = pd.read_csv('../data/sliding01.csv', sep=',', index_col=0)
df02 = pd.read_csv('../data/sliding02_shots.csv', sep=',', index_col=0)
df03 = pd.read_csv('../data/sliding03_shots_extra.csv', sep=',', index_col=0)
df04 = pd.read_csv('../data/sliding04_shots_possession_extra.csv', sep=',', index_col=0)

n01 = normalize_and_encode(df01)
n02 = normalize_and_encode(df02)
n03 = normalize_and_encode(df03)
n04 = normalize_and_encode(df04)

train01, test01 = train_test_split(n01, test_size=0.1)
print(len(train01), 'train examples')
print(len(test01), 'test examples')

train02, test02 = train_test_split(n02, test_size=0.1)
print(len(train02), 'train examples')
print(len(test02), 'test examples')

train03, test03 = train_test_split(n03, test_size=0.1)
print(len(train03), 'train examples')
print(len(test03), 'test examples')

train04, test04 = train_test_split(n04, test_size=0.1)
print(len(train04), 'train examples')
print(len(test04), 'test examples')


train_X01,train_y01 = get_X_and_y(train01)
train_X02,train_y02 = get_X_and_y(train02)
train_X03,train_y03 = get_X_and_y(train03)
train_X04,train_y04 = get_X_and_y(train04)

test_X01,test_y01 = get_X_and_y(test01)
test_X02,test_y02 = get_X_and_y(test02)
test_X03,test_y03 = get_X_and_y(test03)
test_X04,test_y04 = get_X_and_y(test04)

18740 train examples
2083 test examples
6329 train examples
704 test examples
6329 train examples
704 test examples
6296 train examples
700 test examples


In [3]:
n01

,result,odds-home,odds-draw,odds-away,home-wins,home-draws,home-losses,home-goals,home-opposition-goals,away-wins,away-draws,away-losses,away-goals,away-opposition-goals
0,2,0.049957,0.165301,0.330601,0.183667,0.110200,0.073467,0.514268,0.367334,0.073467,0.073467,0.220401,0.293868,0.514268
1,1,0.077897,0.103862,0.121172,0.103862,0.138483,0.103862,0.553931,0.415448,0.138483,0.103862,0.103862,0.484690,0.415448
2,1,0.109311,0.117119,0.105407,0.195198,0.078079,0.117119,0.390396,0.312317,0.156158,0.156158,0.078079,0.585594,0.507514
3,0,0.068789,0.117049,0.155945,0.180075,0.108045,0.072030,0.648271,0.396166,0.108045,0.108045,0.144060,0.360151,0.396166
4,2,0.108097,0.154424,0.205899,0.102949,0.205899,0.205899,0.308848,0.463272,0.051475,0.308848,0.154424,0.360322,0.514746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20818,2,0.162536,0.123527,0.055262,0.097521,0.065014,0.162536,0.260057,0.487607,0.065014,0.130029,0.130029,0.325071,0.682650
20819,2,0.061498,0.110696,0.113771,0.092247,0.092247,0.122995,0.307489,0.584228,0.030749,0.122995,0.153744,0.215242,0.645726
20820,2,0.073697,0.153535,0.184242,0.122828,0.163770,0.122828,0.409426,0.368484,0.204713,0.081885,0.122828,0.614139,0.368484
20821,2,0.044350,0.175065,0.300111,0.066691,0.100037,0.166729,0.400149,0.466840,0.100037,0.100037,0.133383,0.366803,0.533532


In [4]:
n02

,result,odds-home,odds-draw,odds-away,home-wins,home-draws,home-losses,home-goals,home-opposition-goals,home-shots,home-shots_on_target,home-opposition_shots,home-opposition_shots_on_target,away-wins,away-draws,away-losses,away-goals,away-opposition-goals,away-shots,away-shots_on_target,away-opposition_shots,away-opposition_shots_on_target
0,2,0.012482,0.011769,0.007489,0.003566,0.010699,0.021398,0.039230,0.057061,0.488587,0.238944,0.417260,0.189015,0.028531,0.007133,0.000000,0.053495,0.021398,0.574178,0.278173,0.256775,0.106990
1,1,0.009236,0.012191,0.010640,0.011083,0.003694,0.022166,0.029555,0.059110,0.495044,0.236439,0.557848,0.284465,0.011083,0.011083,0.014777,0.040638,0.066498,0.384213,0.162552,0.321409,0.132997
2,0,0.007188,0.012795,0.015805,0.015053,0.007526,0.015053,0.037632,0.056448,0.451585,0.218266,0.466638,0.210740,0.007526,0.007526,0.022579,0.041395,0.056448,0.504270,0.222029,0.376321,0.173108
3,2,0.010289,0.010289,0.007281,0.015829,0.006332,0.009497,0.069647,0.037989,0.560339,0.259592,0.234266,0.117133,0.018995,0.009497,0.003166,0.060149,0.025326,0.535013,0.300747,0.357730,0.183614
4,2,0.004077,0.020384,0.064551,0.023782,0.006795,0.003397,0.050961,0.027179,0.546982,0.244613,0.251408,0.105320,0.010192,0.006795,0.016987,0.033974,0.057756,0.485829,0.234421,0.455252,0.234421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7028,2,0.020289,0.015420,0.006898,0.016231,0.008116,0.016231,0.040578,0.048694,0.324626,0.174487,0.474766,0.243470,0.008116,0.016231,0.016231,0.040578,0.085214,0.482881,0.235354,0.454477,0.263759
7029,2,0.009960,0.017929,0.018427,0.014941,0.009960,0.024901,0.044822,0.104584,0.443238,0.234070,0.458179,0.229089,0.004980,0.019921,0.024901,0.034861,0.104584,0.517941,0.273911,0.313753,0.129485
7030,2,0.007617,0.015870,0.019044,0.012696,0.012696,0.016928,0.033855,0.033855,0.516293,0.249683,0.389336,0.211596,0.021160,0.008464,0.012696,0.063479,0.038087,0.355481,0.181972,0.499366,0.236987
7031,2,0.005271,0.020806,0.035667,0.007926,0.011889,0.019815,0.047556,0.055483,0.491416,0.245708,0.392341,0.198152,0.011889,0.011889,0.015852,0.043593,0.067372,0.408193,0.210041,0.483490,0.214004


In [5]:
n03

,result,odds-home,odds-draw,odds-away,home-wins,home-draws,home-losses,home-goals,home-opposition-goals,home-shots,home-shots_on_target,home-opposition_shots,home-opposition_shots_on_target,away-wins,away-draws,away-losses,away-goals,away-opposition-goals,away-shots,away-shots_on_target,away-opposition_shots,away-opposition_shots_on_target,home_shot_accuracy,home_shot_efficiency,home_opposition_shot_accuracy,home_opposition_shot_efficiency,away_shot_accuracy,away_shot_efficiency,away_opposition_shot_accuracy,away_opposition_shot_efficiency
0,2,0.012482,0.011769,0.007489,0.003566,0.010699,0.021398,0.039229,0.057061,0.488583,0.238942,0.417257,0.189014,0.028530,0.007133,0.000000,0.053495,0.021398,0.574174,0.278171,0.256774,0.106989,0.001744,0.000586,0.001616,0.001077,0.001728,0.000686,0.001486,0.000713
1,1,0.009236,0.012191,0.010640,0.011083,0.003694,0.022166,0.029555,0.059109,0.495039,0.236437,0.557843,0.284463,0.011083,0.011083,0.014777,0.040638,0.066498,0.384210,0.162550,0.321406,0.132996,0.001764,0.000462,0.001884,0.000768,0.001563,0.000924,0.001529,0.001847
2,0,0.007188,0.012795,0.015805,0.015053,0.007526,0.015053,0.037632,0.056448,0.451581,0.218264,0.466634,0.210738,0.007526,0.007526,0.022579,0.041395,0.056448,0.504266,0.222028,0.376318,0.173106,0.001819,0.000649,0.001699,0.001008,0.001657,0.000702,0.001731,0.001227
3,2,0.010289,0.010289,0.007281,0.015829,0.006331,0.009497,0.069646,0.037989,0.560335,0.259590,0.234264,0.117132,0.018994,0.009497,0.003166,0.060149,0.025326,0.535009,0.300745,0.357728,0.183613,0.001467,0.000849,0.001583,0.001027,0.001780,0.000633,0.001625,0.000437
4,2,0.004077,0.020384,0.064550,0.023782,0.006795,0.003397,0.050961,0.027179,0.546979,0.244612,0.251406,0.105319,0.010192,0.006795,0.016987,0.033974,0.057756,0.485826,0.234419,0.455249,0.234419,0.001519,0.000708,0.001423,0.000877,0.001639,0.000492,0.001749,0.000837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7028,2,0.020289,0.015420,0.006898,0.016231,0.008116,0.016231,0.040578,0.048693,0.324623,0.174485,0.474761,0.243467,0.008116,0.016231,0.016231,0.040578,0.085213,0.482876,0.235351,0.454472,0.263756,0.002181,0.000944,0.002081,0.000812,0.001978,0.000700,0.002355,0.001311
7029,2,0.009960,0.017928,0.018426,0.014940,0.009960,0.024900,0.044821,0.104582,0.443228,0.234064,0.458168,0.229084,0.004980,0.019920,0.024900,0.034861,0.104582,0.517929,0.273905,0.313745,0.129482,0.002630,0.000954,0.002490,0.002274,0.002634,0.000634,0.002055,0.004022
7030,2,0.007617,0.015870,0.019043,0.012696,0.012696,0.016927,0.033855,0.033855,0.516288,0.249680,0.389332,0.211593,0.021159,0.008464,0.012696,0.063478,0.038087,0.355477,0.181970,0.499360,0.236985,0.002047,0.000574,0.002300,0.000677,0.002166,0.001476,0.002008,0.000680
7031,2,0.005271,0.020806,0.035667,0.007926,0.011889,0.019815,0.047556,0.055482,0.491412,0.245706,0.392337,0.198150,0.011889,0.011889,0.015852,0.043593,0.067371,0.408189,0.210039,0.483486,0.214002,0.001981,0.000767,0.002002,0.001110,0.002039,0.000823,0.001754,0.001248


In [6]:
n04

,result,odds-home,odds-draw,odds-away,home-wins,home-draws,home-losses,home-goals,home-opposition-goals,home-shots,home-shots_on_target,home-possession,home-opposition_shots,home-opposition_shots_on_target,home-opposition_possession,away-wins,away-draws,away-losses,away-goals,away-opposition-goals,away-shots,away-shots_on_target,away-possession,away-opposition_shots,away-opposition_shots_on_target,away-opposition_possession,home_shot_accuracy,home_shot_efficiency,home_opposition_shot_accuracy,home_opposition_shot_efficiency,away_shot_accuracy,away_shot_efficiency,away_opposition_shot_accuracy,away_opposition_shot_efficiency
0,2,0.012482,0.011769,0.007489,0.003566,0.010699,0.021398,0.039229,0.057060,0.488580,0.238941,0.001887,0.417255,0.189013,0.001680,0.028530,0.007133,0.000000,0.053494,0.021398,0.574171,0.278170,0.001886,0.256772,0.106988,0.001680,0.001744,0.000586,0.001615,0.001077,0.001728,0.000686,0.001486,0.000713
1,2,0.010056,0.010056,0.007116,0.015470,0.006188,0.009282,0.058786,0.037128,0.584770,0.275368,0.001837,0.213488,0.111385,0.001257,0.018564,0.009282,0.003094,0.058786,0.024752,0.522890,0.293932,0.001865,0.349625,0.179453,0.001229,0.001457,0.000661,0.001614,0.001031,0.001739,0.000619,0.001588,0.000427
2,2,0.004131,0.020657,0.065414,0.024100,0.006886,0.003443,0.051643,0.027543,0.554301,0.247886,0.001803,0.254772,0.106729,0.001640,0.006886,0.006886,0.020657,0.030986,0.061972,0.454458,0.213458,0.001607,0.478558,0.244443,0.001836,0.001540,0.000717,0.001442,0.000888,0.001617,0.000500,0.001759,0.000873
3,0,0.008720,0.012090,0.010991,0.010991,0.010991,0.014655,0.032974,0.040301,0.373702,0.172196,0.001766,0.487279,0.282109,0.001897,0.010991,0.007327,0.018319,0.032974,0.058620,0.424995,0.197842,0.001646,0.479951,0.234480,0.002018,0.001688,0.000702,0.002121,0.000523,0.001706,0.000611,0.001790,0.000916
4,0,0.008796,0.010995,0.009473,0.013532,0.003383,0.016916,0.050747,0.054130,0.422888,0.213136,0.001622,0.517615,0.246967,0.001761,0.013532,0.010149,0.010149,0.043980,0.054130,0.348460,0.172538,0.001621,0.466868,0.257116,0.001762,0.001705,0.000806,0.001614,0.000742,0.001675,0.000862,0.001863,0.000712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6991,2,0.020289,0.015419,0.006898,0.016231,0.008115,0.016231,0.040577,0.048693,0.324620,0.174483,0.001846,0.474757,0.243465,0.002212,0.008115,0.016231,0.016231,0.040577,0.085213,0.482872,0.235349,0.001733,0.454468,0.263754,0.002325,0.002181,0.000944,0.002081,0.000812,0.001978,0.000700,0.002355,0.001311
6992,2,0.009960,0.017928,0.018426,0.014940,0.009960,0.024900,0.044820,0.104580,0.443222,0.234061,0.002136,0.458162,0.229081,0.002844,0.004980,0.019920,0.024900,0.034860,0.104580,0.517922,0.273901,0.002297,0.313741,0.129481,0.002683,0.002630,0.000954,0.002490,0.002273,0.002634,0.000634,0.002055,0.004022
6993,2,0.007617,0.015869,0.019043,0.012695,0.012695,0.016927,0.033855,0.033855,0.516283,0.249678,0.002087,0.389328,0.211591,0.002144,0.021159,0.008464,0.012695,0.063477,0.038086,0.355474,0.181969,0.002306,0.499356,0.236982,0.001925,0.002047,0.000574,0.002300,0.000677,0.002166,0.001476,0.002008,0.000680
6994,2,0.005271,0.020806,0.035667,0.007926,0.011889,0.019815,0.047556,0.055482,0.491408,0.245704,0.002312,0.392334,0.198148,0.001651,0.011889,0.011889,0.015852,0.043593,0.067370,0.408186,0.210037,0.001906,0.483482,0.214000,0.002057,0.001981,0.000767,0.002001,0.001110,0.002039,0.000823,0.001754,0.001248


In [7]:
#number of columns for training data
print(train_X01.shape[1])
print(train_X02.shape[1])
print(train_X03.shape[1])
print(train_X04.shape[1])

13
21
29
33


In [8]:
model01 = tf.keras.Sequential([
  layers.Dense(13, activation='relu',input_shape=(train_X01.shape[1],)), # 13 features
  layers.Dense(16, activation='relu'),
  layers.Dense(8, activation='relu'),
  layers.Dense(3, activation='softmax')
])
#model01.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )
#model01.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False) , metrics = ['accuracy'] )
#model01.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) , metrics = ['accuracy'] )
model01.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adagrad(learning_rate=0.01) , metrics = ['accuracy'] )

model02 = tf.keras.Sequential([
  layers.Dense(21, activation='relu',input_shape=(train_X02.shape[1],)), # 21 features
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(16, activation='relu'),
  layers.Dense(8, activation='relu'),
  layers.Dense(3, activation='softmax')
])
#model02.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )
#model02.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False) , metrics = ['accuracy'] )
#model02.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) , metrics = ['accuracy'] )
model02.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adagrad(learning_rate=0.01) , metrics = ['accuracy'] )

model03 = tf.keras.Sequential([
  layers.Dense(29, activation='relu',input_shape=(train_X03.shape[1],)), # 29 features
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(16, activation='relu'),
  layers.Dense(8, activation='relu'),
  layers.Dense(3, activation='softmax')
])
#model03.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )
#model03.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False) , metrics = ['accuracy'] )
#model03.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) , metrics = ['accuracy'] )
model03.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adagrad(learning_rate=0.01) , metrics = ['accuracy'] )

model04 = tf.keras.Sequential([
  layers.Dense(33, activation='relu',input_shape=(train_X04.shape[1],)), # 33 features
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(16, activation='relu'),
  layers.Dense(8, activation='relu'),
  layers.Dense(3, activation='softmax')
])
#model03.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )
#model03.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False) , metrics = ['accuracy'] )
#model03.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) , metrics = ['accuracy'] )
model04.compile(loss = 'sparse_categorical_crossentropy' , optimizer = keras.optimizers.Adagrad(learning_rate=0.01) , metrics = ['accuracy'] )


early_stoping = EarlyStopping(patience=50)
validation_split = 0.2
epochs=500

In [9]:
model01.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 13)                182       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                224       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27        
Total params: 569
Trainable params: 569
Non-trainable params: 0
_________________________________________________________________


In [10]:
model01.fit(train_X01, train_y01, validation_split=validation_split,epochs=epochs,callbacks=[early_stoping])

Train on 14992 samples, validate on 3748 samples
Epoch 1/500
14992/14992 [==============================] - 4s 246us/sample - loss: 1.0439 - accuracy: 0.4747 - val_loss: 1.0256 - val_accuracy: 0.4995
Epoch 2/500
14992/14992 [==============================] - 2s 117us/sample - loss: 1.0125 - accuracy: 0.5078 - val_loss: 1.0074 - val_accuracy: 0.5035
Epoch 3/500
14992/14992 [==============================] - 2s 112us/sample - loss: 0.9999 - accuracy: 0.5135 - val_loss: 0.9998 - val_accuracy: 0.5029
Epoch 4/500
14992/14992 [==============================] - 1s 77us/sample - loss: 0.9938 - accuracy: 0.5154 - val_loss: 0.9958 - val_accuracy: 0.5061
Epoch 5/500
14992/14992 [==============================] - 1s 78us/sample - loss: 0.9902 - accuracy: 0.5175 - val_loss: 0.9926 - val_accuracy: 0.5051
Epoch 6/500
14992/14992 [==============================] - 1s 77us/sample - loss: 0.9877 - accuracy: 0.5190 - val_loss: 0.9898 - val_accuracy: 0.5117
Epoch 7/500
14992/14992 [=======================

Epoch 55/500
14992/14992 [==============================] - 1s 97us/sample - loss: 0.9708 - accuracy: 0.5324 - val_loss: 0.9767 - val_accuracy: 0.5221
Epoch 56/500
14992/14992 [==============================] - 1s 86us/sample - loss: 0.9707 - accuracy: 0.5301 - val_loss: 0.9769 - val_accuracy: 0.5251
Epoch 57/500
14992/14992 [==============================] - 1s 84us/sample - loss: 0.9706 - accuracy: 0.5314 - val_loss: 0.9768 - val_accuracy: 0.5235
Epoch 58/500
14992/14992 [==============================] - 1s 98us/sample - loss: 0.9707 - accuracy: 0.5309 - val_loss: 0.9767 - val_accuracy: 0.5224
Epoch 59/500
14992/14992 [==============================] - 1s 91us/sample - loss: 0.9706 - accuracy: 0.5304 - val_loss: 0.9779 - val_accuracy: 0.5235
Epoch 60/500
14992/14992 [==============================] - 1s 95us/sample - loss: 0.9708 - accuracy: 0.5309 - val_loss: 0.9767 - val_accuracy: 0.5251
Epoch 61/500
14992/14992 [==============================] - 2s 113us/sample - loss: 0.9707 - a

14992/14992 [==============================] - 1s 82us/sample - loss: 0.9702 - accuracy: 0.5320 - val_loss: 0.9767 - val_accuracy: 0.5235
Epoch 110/500
14992/14992 [==============================] - 1s 80us/sample - loss: 0.9702 - accuracy: 0.5318 - val_loss: 0.9767 - val_accuracy: 0.5245
Epoch 111/500
14992/14992 [==============================] - 1s 78us/sample - loss: 0.9703 - accuracy: 0.5317 - val_loss: 0.9768 - val_accuracy: 0.5243
Epoch 112/500
14992/14992 [==============================] - 1s 78us/sample - loss: 0.9703 - accuracy: 0.5308 - val_loss: 0.9769 - val_accuracy: 0.5261
Epoch 113/500
14992/14992 [==============================] - 1s 97us/sample - loss: 0.9703 - accuracy: 0.5314 - val_loss: 0.9767 - val_accuracy: 0.5232
Epoch 114/500
14992/14992 [==============================] - 3s 176us/sample - loss: 0.9703 - accuracy: 0.5321 - val_loss: 0.9772 - val_accuracy: 0.5256
Epoch 115/500
14992/14992 [==============================] - 2s 161us/sample - loss: 0.9703 - accurac

In [11]:
model02.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 21)                462       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                1408      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 27        
Total params: 4,641
Trainable params: 4,641
Non-trainable params: 0
____________________________________________________

In [12]:
model02.fit(train_X02, train_y02, validation_split=validation_split,epochs=epochs,callbacks=[early_stoping])

Train on 5063 samples, validate on 1266 samples
Epoch 1/500
5063/5063 [==============================] - 2s 338us/sample - loss: 1.0645 - accuracy: 0.4679 - val_loss: 1.0769 - val_accuracy: 0.4368
Epoch 2/500
5063/5063 [==============================] - 0s 98us/sample - loss: 1.0587 - accuracy: 0.4669 - val_loss: 1.0753 - val_accuracy: 0.4368
Epoch 3/500
5063/5063 [==============================] - 0s 97us/sample - loss: 1.0579 - accuracy: 0.4669 - val_loss: 1.0749 - val_accuracy: 0.4368
Epoch 4/500
5063/5063 [==============================] - 1s 100us/sample - loss: 1.0566 - accuracy: 0.4669 - val_loss: 1.0736 - val_accuracy: 0.4368
Epoch 5/500
5063/5063 [==============================] - 1s 100us/sample - loss: 1.0513 - accuracy: 0.4669 - val_loss: 1.0662 - val_accuracy: 0.4368
Epoch 6/500
5063/5063 [==============================] - 1s 99us/sample - loss: 1.0467 - accuracy: 0.4669 - val_loss: 1.0615 - val_accuracy: 0.4368
Epoch 7/500
5063/5063 [==============================] - 1s 9

5063/5063 [==============================] - 1s 101us/sample - loss: 0.9936 - accuracy: 0.5246 - val_loss: 1.0075 - val_accuracy: 0.5008
Epoch 56/500
5063/5063 [==============================] - 1s 105us/sample - loss: 0.9927 - accuracy: 0.5262 - val_loss: 1.0110 - val_accuracy: 0.5032
Epoch 57/500
5063/5063 [==============================] - 1s 125us/sample - loss: 0.9930 - accuracy: 0.5236 - val_loss: 1.0076 - val_accuracy: 0.5024
Epoch 58/500
5063/5063 [==============================] - 1s 136us/sample - loss: 0.9925 - accuracy: 0.5264 - val_loss: 1.0069 - val_accuracy: 0.5047
Epoch 59/500
5063/5063 [==============================] - 1s 115us/sample - loss: 0.9923 - accuracy: 0.5258 - val_loss: 1.0092 - val_accuracy: 0.4984
Epoch 60/500
5063/5063 [==============================] - 1s 136us/sample - loss: 0.9922 - accuracy: 0.5252 - val_loss: 1.0089 - val_accuracy: 0.4992
Epoch 61/500
5063/5063 [==============================] - 1s 100us/sample - loss: 0.9920 - accuracy: 0.5276 - val

Epoch 110/500
5063/5063 [==============================] - 1s 99us/sample - loss: 0.9820 - accuracy: 0.5351 - val_loss: 0.9978 - val_accuracy: 0.5103
Epoch 111/500
5063/5063 [==============================] - 1s 152us/sample - loss: 0.9822 - accuracy: 0.5327 - val_loss: 0.9989 - val_accuracy: 0.5126
Epoch 112/500
5063/5063 [==============================] - 1s 123us/sample - loss: 0.9819 - accuracy: 0.5307 - val_loss: 0.9995 - val_accuracy: 0.5142
Epoch 113/500
5063/5063 [==============================] - 1s 125us/sample - loss: 0.9818 - accuracy: 0.5313 - val_loss: 0.9995 - val_accuracy: 0.5126
Epoch 114/500
5063/5063 [==============================] - 1s 105us/sample - loss: 0.9812 - accuracy: 0.5335 - val_loss: 0.9977 - val_accuracy: 0.5039
Epoch 115/500
5063/5063 [==============================] - 1s 146us/sample - loss: 0.9815 - accuracy: 0.5301 - val_loss: 0.9976 - val_accuracy: 0.5103
Epoch 116/500
5063/5063 [==============================] - 1s 158us/sample - loss: 0.9814 - acc

5063/5063 [==============================] - 1s 100us/sample - loss: 0.9762 - accuracy: 0.5321 - val_loss: 0.9936 - val_accuracy: 0.5087
Epoch 165/500
5063/5063 [==============================] - 0s 96us/sample - loss: 0.9758 - accuracy: 0.5337 - val_loss: 0.9928 - val_accuracy: 0.5111
Epoch 166/500
5063/5063 [==============================] - 1s 99us/sample - loss: 0.9761 - accuracy: 0.5313 - val_loss: 0.9927 - val_accuracy: 0.5095
Epoch 167/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9758 - accuracy: 0.5337 - val_loss: 0.9994 - val_accuracy: 0.5087
Epoch 168/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9757 - accuracy: 0.5355 - val_loss: 0.9932 - val_accuracy: 0.5047
Epoch 169/500
5063/5063 [==============================] - 0s 95us/sample - loss: 0.9756 - accuracy: 0.5357 - val_loss: 0.9932 - val_accuracy: 0.5095
Epoch 170/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9756 - accuracy: 0.5364 - val

5063/5063 [==============================] - 0s 98us/sample - loss: 0.9723 - accuracy: 0.5368 - val_loss: 0.9896 - val_accuracy: 0.5134
Epoch 219/500
5063/5063 [==============================] - 0s 96us/sample - loss: 0.9722 - accuracy: 0.5358 - val_loss: 0.9913 - val_accuracy: 0.5126
Epoch 220/500
5063/5063 [==============================] - 1s 112us/sample - loss: 0.9721 - accuracy: 0.5370 - val_loss: 0.9897 - val_accuracy: 0.5118
Epoch 221/500
5063/5063 [==============================] - 1s 112us/sample - loss: 0.9718 - accuracy: 0.5386 - val_loss: 0.9900 - val_accuracy: 0.5134
Epoch 222/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9715 - accuracy: 0.5368 - val_loss: 0.9903 - val_accuracy: 0.5063
Epoch 223/500
5063/5063 [==============================] - 1s 110us/sample - loss: 0.9720 - accuracy: 0.5349 - val_loss: 0.9906 - val_accuracy: 0.5111
Epoch 224/500
5063/5063 [==============================] - 1s 117us/sample - loss: 0.9719 - accuracy: 0.5366 - 

5063/5063 [==============================] - 1s 137us/sample - loss: 0.9696 - accuracy: 0.5366 - val_loss: 0.9891 - val_accuracy: 0.5134
Epoch 273/500
5063/5063 [==============================] - 1s 144us/sample - loss: 0.9696 - accuracy: 0.5368 - val_loss: 0.9904 - val_accuracy: 0.5103
Epoch 274/500
5063/5063 [==============================] - 1s 148us/sample - loss: 0.9696 - accuracy: 0.5392 - val_loss: 0.9880 - val_accuracy: 0.5150
Epoch 275/500
5063/5063 [==============================] - 1s 133us/sample - loss: 0.9697 - accuracy: 0.5368 - val_loss: 0.9882 - val_accuracy: 0.5150
Epoch 276/500
5063/5063 [==============================] - 1s 141us/sample - loss: 0.9688 - accuracy: 0.5390 - val_loss: 0.9922 - val_accuracy: 0.5103
Epoch 277/500
5063/5063 [==============================] - 1s 143us/sample - loss: 0.9696 - accuracy: 0.5380 - val_loss: 0.9878 - val_accuracy: 0.5134
Epoch 278/500
5063/5063 [==============================] - 1s 103us/sample - loss: 0.9693 - accuracy: 0.5380

5063/5063 [==============================] - 1s 121us/sample - loss: 0.9674 - accuracy: 0.5398 - val_loss: 0.9866 - val_accuracy: 0.5118
Epoch 327/500
5063/5063 [==============================] - 1s 124us/sample - loss: 0.9675 - accuracy: 0.5366 - val_loss: 0.9870 - val_accuracy: 0.5142
Epoch 328/500
5063/5063 [==============================] - 1s 118us/sample - loss: 0.9676 - accuracy: 0.5388 - val_loss: 0.9866 - val_accuracy: 0.5118
Epoch 329/500
5063/5063 [==============================] - 1s 124us/sample - loss: 0.9676 - accuracy: 0.5380 - val_loss: 0.9865 - val_accuracy: 0.5142
Epoch 330/500
5063/5063 [==============================] - 1s 107us/sample - loss: 0.9672 - accuracy: 0.5396 - val_loss: 0.9907 - val_accuracy: 0.5079
Epoch 331/500
5063/5063 [==============================] - 1s 117us/sample - loss: 0.9676 - accuracy: 0.5394 - val_loss: 0.9862 - val_accuracy: 0.5126
Epoch 332/500
5063/5063 [==============================] - 1s 124us/sample - loss: 0.9669 - accuracy: 0.5386

5063/5063 [==============================] - 1s 114us/sample - loss: 0.9657 - accuracy: 0.5388 - val_loss: 0.9853 - val_accuracy: 0.5150
Epoch 381/500
5063/5063 [==============================] - 1s 108us/sample - loss: 0.9655 - accuracy: 0.5384 - val_loss: 0.9857 - val_accuracy: 0.5150
Epoch 382/500
5063/5063 [==============================] - 0s 96us/sample - loss: 0.9655 - accuracy: 0.5402 - val_loss: 0.9866 - val_accuracy: 0.5118
Epoch 383/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9656 - accuracy: 0.5439 - val_loss: 0.9895 - val_accuracy: 0.5071
Epoch 384/500
5063/5063 [==============================] - 1s 133us/sample - loss: 0.9660 - accuracy: 0.5388 - val_loss: 0.9870 - val_accuracy: 0.5118
Epoch 385/500
5063/5063 [==============================] - 1s 122us/sample - loss: 0.9654 - accuracy: 0.5398 - val_loss: 0.9859 - val_accuracy: 0.5134
Epoch 386/500
5063/5063 [==============================] - 1s 108us/sample - loss: 0.9655 - accuracy: 0.5408 -

5063/5063 [==============================] - 1s 128us/sample - loss: 0.9640 - accuracy: 0.5398 - val_loss: 0.9941 - val_accuracy: 0.5063
Epoch 435/500
5063/5063 [==============================] - 1s 121us/sample - loss: 0.9640 - accuracy: 0.5428 - val_loss: 0.9855 - val_accuracy: 0.5087
Epoch 436/500
5063/5063 [==============================] - 1s 121us/sample - loss: 0.9647 - accuracy: 0.5398 - val_loss: 0.9888 - val_accuracy: 0.5063
Epoch 437/500
5063/5063 [==============================] - 1s 115us/sample - loss: 0.9642 - accuracy: 0.5410 - val_loss: 0.9863 - val_accuracy: 0.5126
Epoch 438/500
5063/5063 [==============================] - 1s 110us/sample - loss: 0.9642 - accuracy: 0.5420 - val_loss: 0.9860 - val_accuracy: 0.5126
Epoch 439/500
5063/5063 [==============================] - 1s 111us/sample - loss: 0.9644 - accuracy: 0.5418 - val_loss: 0.9844 - val_accuracy: 0.5126
Epoch 440/500
5063/5063 [==============================] - 1s 109us/sample - loss: 0.9642 - accuracy: 0.5404

5063/5063 [==============================] - 1s 113us/sample - loss: 0.9633 - accuracy: 0.5412 - val_loss: 0.9843 - val_accuracy: 0.5118
Epoch 489/500
5063/5063 [==============================] - 1s 114us/sample - loss: 0.9633 - accuracy: 0.5412 - val_loss: 0.9862 - val_accuracy: 0.5103
Epoch 490/500
5063/5063 [==============================] - 1s 163us/sample - loss: 0.9634 - accuracy: 0.5416 - val_loss: 0.9839 - val_accuracy: 0.5150
Epoch 491/500
5063/5063 [==============================] - 1s 118us/sample - loss: 0.9633 - accuracy: 0.5428 - val_loss: 0.9850 - val_accuracy: 0.5126
Epoch 492/500
5063/5063 [==============================] - 1s 143us/sample - loss: 0.9634 - accuracy: 0.5418 - val_loss: 0.9849 - val_accuracy: 0.5126
Epoch 493/500
5063/5063 [==============================] - 1s 136us/sample - loss: 0.9630 - accuracy: 0.5400 - val_loss: 0.9865 - val_accuracy: 0.5071
Epoch 494/500
5063/5063 [==============================] - 1s 121us/sample - loss: 0.9632 - accuracy: 0.5414

In [13]:
model03.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 29)                870       
_________________________________________________________________
dense_11 (Dense)             (None, 64)                1920      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_13 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 27        
Total params: 5,561
Trainable params: 5,561
Non-trainable params: 0
____________________________________________________

In [14]:
model03.fit(train_X03, train_y03, validation_split=validation_split,epochs=epochs,callbacks=[early_stoping])

Train on 5063 samples, validate on 1266 samples
Epoch 1/500
5063/5063 [==============================] - 2s 374us/sample - loss: 1.0742 - accuracy: 0.4484 - val_loss: 1.0623 - val_accuracy: 0.4645
Epoch 2/500
5063/5063 [==============================] - 1s 123us/sample - loss: 1.0613 - accuracy: 0.4584 - val_loss: 1.0574 - val_accuracy: 0.4645
Epoch 3/500
5063/5063 [==============================] - 1s 121us/sample - loss: 1.0579 - accuracy: 0.4584 - val_loss: 1.0563 - val_accuracy: 0.4645
Epoch 4/500
5063/5063 [==============================] - 1s 106us/sample - loss: 1.0540 - accuracy: 0.4584 - val_loss: 1.0510 - val_accuracy: 0.4645
Epoch 5/500
5063/5063 [==============================] - 1s 113us/sample - loss: 1.0502 - accuracy: 0.4584 - val_loss: 1.0456 - val_accuracy: 0.4645
Epoch 6/500
5063/5063 [==============================] - 1s 122us/sample - loss: 1.0457 - accuracy: 0.4584 - val_loss: 1.0405 - val_accuracy: 0.4645
Epoch 7/500
5063/5063 [==============================] - 1

5063/5063 [==============================] - 1s 100us/sample - loss: 0.9875 - accuracy: 0.5232 - val_loss: 0.9816 - val_accuracy: 0.5150
Epoch 56/500
5063/5063 [==============================] - 0s 96us/sample - loss: 0.9871 - accuracy: 0.5244 - val_loss: 0.9775 - val_accuracy: 0.5340
Epoch 57/500
5063/5063 [==============================] - 1s 100us/sample - loss: 0.9872 - accuracy: 0.5258 - val_loss: 0.9805 - val_accuracy: 0.5142
Epoch 58/500
5063/5063 [==============================] - 1s 99us/sample - loss: 0.9877 - accuracy: 0.5234 - val_loss: 0.9793 - val_accuracy: 0.5237
Epoch 59/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9866 - accuracy: 0.5258 - val_loss: 0.9762 - val_accuracy: 0.5324
Epoch 60/500
5063/5063 [==============================] - 0s 96us/sample - loss: 0.9863 - accuracy: 0.5266 - val_loss: 0.9758 - val_accuracy: 0.5332
Epoch 61/500
5063/5063 [==============================] - 1s 99us/sample - loss: 0.9854 - accuracy: 0.5287 - val_loss

Epoch 110/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9777 - accuracy: 0.5317 - val_loss: 0.9700 - val_accuracy: 0.5261
Epoch 111/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9773 - accuracy: 0.5335 - val_loss: 0.9663 - val_accuracy: 0.5355
Epoch 112/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9777 - accuracy: 0.5303 - val_loss: 0.9659 - val_accuracy: 0.5355
Epoch 113/500
5063/5063 [==============================] - 0s 99us/sample - loss: 0.9771 - accuracy: 0.5360 - val_loss: 0.9664 - val_accuracy: 0.5442
Epoch 114/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9774 - accuracy: 0.5319 - val_loss: 0.9661 - val_accuracy: 0.5348
Epoch 115/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9775 - accuracy: 0.5339 - val_loss: 0.9661 - val_accuracy: 0.5450
Epoch 116/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9769 - accuracy:

5063/5063 [==============================] - 1s 105us/sample - loss: 0.9732 - accuracy: 0.5333 - val_loss: 0.9625 - val_accuracy: 0.5363
Epoch 165/500
5063/5063 [==============================] - 1s 104us/sample - loss: 0.9739 - accuracy: 0.5341 - val_loss: 0.9634 - val_accuracy: 0.5340
Epoch 166/500
5063/5063 [==============================] - 1s 100us/sample - loss: 0.9736 - accuracy: 0.5303 - val_loss: 0.9624 - val_accuracy: 0.5371
Epoch 167/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9731 - accuracy: 0.5349 - val_loss: 0.9622 - val_accuracy: 0.5371
Epoch 168/500
5063/5063 [==============================] - 1s 103us/sample - loss: 0.9735 - accuracy: 0.5370 - val_loss: 0.9623 - val_accuracy: 0.5387
Epoch 169/500
5063/5063 [==============================] - 1s 102us/sample - loss: 0.9737 - accuracy: 0.5364 - val_loss: 0.9622 - val_accuracy: 0.5371
Epoch 170/500
5063/5063 [==============================] - 1s 99us/sample - loss: 0.9739 - accuracy: 0.5349 -

5063/5063 [==============================] - 1s 99us/sample - loss: 0.9716 - accuracy: 0.5343 - val_loss: 0.9606 - val_accuracy: 0.5419
Epoch 219/500
5063/5063 [==============================] - 1s 101us/sample - loss: 0.9713 - accuracy: 0.5358 - val_loss: 0.9612 - val_accuracy: 0.5387
Epoch 220/500
5063/5063 [==============================] - 0s 96us/sample - loss: 0.9713 - accuracy: 0.5341 - val_loss: 0.9606 - val_accuracy: 0.5371
Epoch 221/500
5063/5063 [==============================] - 1s 102us/sample - loss: 0.9711 - accuracy: 0.5372 - val_loss: 0.9613 - val_accuracy: 0.5395
Epoch 222/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9713 - accuracy: 0.5362 - val_loss: 0.9605 - val_accuracy: 0.5403
Epoch 223/500
5063/5063 [==============================] - 0s 99us/sample - loss: 0.9706 - accuracy: 0.5392 - val_loss: 0.9616 - val_accuracy: 0.5355
Epoch 224/500
5063/5063 [==============================] - 0s 97us/sample - loss: 0.9710 - accuracy: 0.5357 - va

5063/5063 [==============================] - 1s 124us/sample - loss: 0.9695 - accuracy: 0.5366 - val_loss: 0.9605 - val_accuracy: 0.5355
Epoch 273/500
5063/5063 [==============================] - 1s 137us/sample - loss: 0.9693 - accuracy: 0.5355 - val_loss: 0.9593 - val_accuracy: 0.5387
Epoch 274/500
5063/5063 [==============================] - 1s 130us/sample - loss: 0.9690 - accuracy: 0.5378 - val_loss: 0.9592 - val_accuracy: 0.5379
Epoch 275/500
5063/5063 [==============================] - 1s 121us/sample - loss: 0.9693 - accuracy: 0.5410 - val_loss: 0.9592 - val_accuracy: 0.5403
Epoch 276/500
5063/5063 [==============================] - 1s 151us/sample - loss: 0.9686 - accuracy: 0.5345 - val_loss: 0.9602 - val_accuracy: 0.5355
Epoch 277/500
5063/5063 [==============================] - 1s 139us/sample - loss: 0.9695 - accuracy: 0.5351 - val_loss: 0.9595 - val_accuracy: 0.5395
Epoch 278/500
5063/5063 [==============================] - 1s 150us/sample - loss: 0.9691 - accuracy: 0.5382

5063/5063 [==============================] - 1s 155us/sample - loss: 0.9678 - accuracy: 0.5378 - val_loss: 0.9589 - val_accuracy: 0.5355
Epoch 327/500
5063/5063 [==============================] - 1s 144us/sample - loss: 0.9680 - accuracy: 0.5384 - val_loss: 0.9589 - val_accuracy: 0.5340
Epoch 328/500
5063/5063 [==============================] - 1s 129us/sample - loss: 0.9679 - accuracy: 0.5388 - val_loss: 0.9588 - val_accuracy: 0.5355
Epoch 329/500
5063/5063 [==============================] - 1s 131us/sample - loss: 0.9674 - accuracy: 0.5378 - val_loss: 0.9585 - val_accuracy: 0.5371
Epoch 330/500
5063/5063 [==============================] - 1s 139us/sample - loss: 0.9676 - accuracy: 0.5378 - val_loss: 0.9591 - val_accuracy: 0.5355
Epoch 331/500
5063/5063 [==============================] - 1s 130us/sample - loss: 0.9679 - accuracy: 0.5368 - val_loss: 0.9586 - val_accuracy: 0.5395
Epoch 332/500
5063/5063 [==============================] - 1s 164us/sample - loss: 0.9676 - accuracy: 0.5347

5063/5063 [==============================] - 1s 110us/sample - loss: 0.9662 - accuracy: 0.5382 - val_loss: 0.9586 - val_accuracy: 0.5348
Epoch 381/500
5063/5063 [==============================] - 1s 106us/sample - loss: 0.9668 - accuracy: 0.5374 - val_loss: 0.9600 - val_accuracy: 0.5332
Epoch 382/500
5063/5063 [==============================] - 1s 107us/sample - loss: 0.9666 - accuracy: 0.5384 - val_loss: 0.9594 - val_accuracy: 0.5348
Epoch 383/500
5063/5063 [==============================] - 1s 115us/sample - loss: 0.9663 - accuracy: 0.5364 - val_loss: 0.9602 - val_accuracy: 0.5411
Epoch 384/500
5063/5063 [==============================] - 1s 110us/sample - loss: 0.9669 - accuracy: 0.5370 - val_loss: 0.9624 - val_accuracy: 0.5261
Epoch 385/500
5063/5063 [==============================] - 1s 119us/sample - loss: 0.9665 - accuracy: 0.5408 - val_loss: 0.9597 - val_accuracy: 0.5355
Epoch 386/500
5063/5063 [==============================] - 1s 118us/sample - loss: 0.9668 - accuracy: 0.5392

5063/5063 [==============================] - 1s 99us/sample - loss: 0.9657 - accuracy: 0.5380 - val_loss: 0.9589 - val_accuracy: 0.5340
Epoch 435/500
5063/5063 [==============================] - 0s 98us/sample - loss: 0.9657 - accuracy: 0.5388 - val_loss: 0.9585 - val_accuracy: 0.5340
Epoch 436/500
5063/5063 [==============================] - 1s 104us/sample - loss: 0.9653 - accuracy: 0.5382 - val_loss: 0.9588 - val_accuracy: 0.5371
Epoch 437/500
5063/5063 [==============================] - 1s 107us/sample - loss: 0.9656 - accuracy: 0.5408 - val_loss: 0.9597 - val_accuracy: 0.5316
Epoch 438/500
5063/5063 [==============================] - 1s 100us/sample - loss: 0.9648 - accuracy: 0.5394 - val_loss: 0.9614 - val_accuracy: 0.5261
Epoch 439/500
5063/5063 [==============================] - 1s 101us/sample - loss: 0.9647 - accuracy: 0.5406 - val_loss: 0.9579 - val_accuracy: 0.5355
Epoch 440/500
5063/5063 [==============================] - 1s 103us/sample - loss: 0.9659 - accuracy: 0.5394 -

5063/5063 [==============================] - 1s 114us/sample - loss: 0.9650 - accuracy: 0.5388 - val_loss: 0.9592 - val_accuracy: 0.5324
Epoch 489/500
5063/5063 [==============================] - 1s 111us/sample - loss: 0.9652 - accuracy: 0.5400 - val_loss: 0.9579 - val_accuracy: 0.5387


In [15]:
model04.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 33)                1122      
_________________________________________________________________
dense_17 (Dense)             (None, 64)                2176      
_________________________________________________________________
dense_18 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_19 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_20 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 27        
Total params: 6,069
Trainable params: 6,069
Non-trainable params: 0
____________________________________________________

In [ ]:
model04.fit(train_X04, train_y04, validation_split=validation_split,epochs=epochs,callbacks=[early_stoping])

Train on 5036 samples, validate on 1260 samples
Epoch 1/500
5036/5036 [==============================] - 2s 413us/sample - loss: 1.0673 - accuracy: 0.4591 - val_loss: 1.0614 - val_accuracy: 0.4627
Epoch 2/500
5036/5036 [==============================] - 1s 117us/sample - loss: 1.0613 - accuracy: 0.4591 - val_loss: 1.0589 - val_accuracy: 0.4627
Epoch 3/500
5036/5036 [==============================] - 1s 150us/sample - loss: 1.0591 - accuracy: 0.4591 - val_loss: 1.0560 - val_accuracy: 0.4627
Epoch 4/500
5036/5036 [==============================] - 1s 122us/sample - loss: 1.0546 - accuracy: 0.4591 - val_loss: 1.0505 - val_accuracy: 0.4627
Epoch 5/500
5036/5036 [==============================] - 1s 101us/sample - loss: 1.0502 - accuracy: 0.4591 - val_loss: 1.0442 - val_accuracy: 0.4627
Epoch 6/500
5036/5036 [==============================] - 1s 152us/sample - loss: 1.0443 - accuracy: 0.4591 - val_loss: 1.0348 - val_accuracy: 0.4627
Epoch 7/500
5036/5036 [==============================] - 1

5036/5036 [==============================] - 1s 108us/sample - loss: 0.9840 - accuracy: 0.5276 - val_loss: 0.9779 - val_accuracy: 0.5198
Epoch 56/500
5036/5036 [==============================] - 1s 115us/sample - loss: 0.9835 - accuracy: 0.5282 - val_loss: 0.9775 - val_accuracy: 0.5238
Epoch 57/500
5036/5036 [==============================] - 1s 132us/sample - loss: 0.9844 - accuracy: 0.5264 - val_loss: 0.9780 - val_accuracy: 0.5294
Epoch 58/500
5036/5036 [==============================] - 1s 130us/sample - loss: 0.9838 - accuracy: 0.5262 - val_loss: 0.9784 - val_accuracy: 0.5302
Epoch 59/500
5036/5036 [==============================] - 1s 114us/sample - loss: 0.9839 - accuracy: 0.5278 - val_loss: 0.9900 - val_accuracy: 0.5127
Epoch 60/500
5036/5036 [==============================] - 1s 112us/sample - loss: 0.9837 - accuracy: 0.5274 - val_loss: 0.9786 - val_accuracy: 0.5206
Epoch 61/500
5036/5036 [==============================] - 1s 115us/sample - loss: 0.9839 - accuracy: 0.5284 - val

5036/5036 [==============================] - 1s 126us/sample - loss: 0.9770 - accuracy: 0.5340 - val_loss: 0.9757 - val_accuracy: 0.5238
Epoch 110/500
5036/5036 [==============================] - 1s 141us/sample - loss: 0.9770 - accuracy: 0.5326 - val_loss: 0.9841 - val_accuracy: 0.5151
Epoch 111/500
5036/5036 [==============================] - 1s 159us/sample - loss: 0.9772 - accuracy: 0.5296 - val_loss: 0.9775 - val_accuracy: 0.5246
Epoch 112/500
5036/5036 [==============================] - 1s 141us/sample - loss: 0.9763 - accuracy: 0.5302 - val_loss: 0.9757 - val_accuracy: 0.5214
Epoch 113/500
5036/5036 [==============================] - 1s 129us/sample - loss: 0.9764 - accuracy: 0.5320 - val_loss: 0.9812 - val_accuracy: 0.5198
Epoch 114/500
5036/5036 [==============================] - 1s 113us/sample - loss: 0.9766 - accuracy: 0.5304 - val_loss: 0.9761 - val_accuracy: 0.5270
Epoch 115/500
5036/5036 [==============================] - 1s 122us/sample - loss: 0.9765 - accuracy: 0.5306

5036/5036 [==============================] - 1s 122us/sample - loss: 0.9732 - accuracy: 0.5340 - val_loss: 0.9791 - val_accuracy: 0.5183
Epoch 164/500
5036/5036 [==============================] - 1s 116us/sample - loss: 0.9733 - accuracy: 0.5326 - val_loss: 0.9761 - val_accuracy: 0.5254
Epoch 165/500
5036/5036 [==============================] - 1s 125us/sample - loss: 0.9733 - accuracy: 0.5353 - val_loss: 0.9783 - val_accuracy: 0.5230
Epoch 166/500
5036/5036 [==============================] - 1s 112us/sample - loss: 0.9725 - accuracy: 0.5330 - val_loss: 0.9758 - val_accuracy: 0.5270
Epoch 167/500
5036/5036 [==============================] - 1s 120us/sample - loss: 0.9729 - accuracy: 0.5328 - val_loss: 0.9749 - val_accuracy: 0.5246
Epoch 168/500
5036/5036 [==============================] - 1s 127us/sample - loss: 0.9727 - accuracy: 0.5365 - val_loss: 0.9837 - val_accuracy: 0.5103
Epoch 169/500
5036/5036 [==============================] - 1s 133us/sample - loss: 0.9733 - accuracy: 0.5347

5036/5036 [==============================] - 1s 131us/sample - loss: 0.9709 - accuracy: 0.5336 - val_loss: 0.9746 - val_accuracy: 0.5270
Epoch 218/500
5036/5036 [==============================] - 1s 138us/sample - loss: 0.9705 - accuracy: 0.5347 - val_loss: 0.9774 - val_accuracy: 0.5206
Epoch 219/500
5036/5036 [==============================] - 1s 112us/sample - loss: 0.9709 - accuracy: 0.5367 - val_loss: 0.9770 - val_accuracy: 0.5222
Epoch 220/500
5036/5036 [==============================] - 1s 101us/sample - loss: 0.9705 - accuracy: 0.5347 - val_loss: 0.9749 - val_accuracy: 0.5262
Epoch 221/500
5036/5036 [==============================] - 1s 100us/sample - loss: 0.9703 - accuracy: 0.5351 - val_loss: 0.9748 - val_accuracy: 0.5286
Epoch 222/500
5036/5036 [==============================] - 0s 99us/sample - loss: 0.9709 - accuracy: 0.5346 - val_loss: 0.9747 - val_accuracy: 0.5286
Epoch 223/500
5036/5036 [==============================] - 0s 97us/sample - loss: 0.9706 - accuracy: 0.5365 -

5036/5036 [==============================] - 1s 113us/sample - loss: 0.9681 - accuracy: 0.5381 - val_loss: 0.9759 - val_accuracy: 0.5206
Epoch 272/500
5036/5036 [==============================] - 1s 104us/sample - loss: 0.9691 - accuracy: 0.5355 - val_loss: 0.9759 - val_accuracy: 0.5222
Epoch 273/500
5036/5036 [==============================] - 1s 100us/sample - loss: 0.9689 - accuracy: 0.5369 - val_loss: 0.9805 - val_accuracy: 0.5135
Epoch 274/500
5036/5036 [==============================] - 0s 99us/sample - loss: 0.9692 - accuracy: 0.5377 - val_loss: 0.9772 - val_accuracy: 0.5159
Epoch 275/500
5036/5036 [==============================] - 0s 99us/sample - loss: 0.9690 - accuracy: 0.5367 - val_loss: 0.9761 - val_accuracy: 0.5206
Epoch 276/500
5036/5036 [==============================] - 1s 100us/sample - loss: 0.9686 - accuracy: 0.5367 - val_loss: 0.9753 - val_accuracy: 0.5262
Epoch 277/500
5036/5036 [==============================] - 0s 98us/sample - loss: 0.9692 - accuracy: 0.5377 - 

In [ ]:
score = model01.evaluate(test_X01, test_y01, verbose=3)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

plt.plot(model01.history.history['accuracy'])
plt.plot(model01.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(model01.history.history['loss'])
plt.plot(model01.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
score = model02.evaluate(test_X02, test_y02, verbose=0)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

plt.plot(model02.history.history['accuracy'])
plt.plot(model02.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(model02.history.history['loss'])
plt.plot(model02.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
score = model03.evaluate(test_X03, test_y03, verbose=0)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

plt.plot(model03.history.history['accuracy'])
plt.plot(model03.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(model03.history.history['loss'])
plt.plot(model03.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [ ]:
score = model04.evaluate(test_X04, test_y04, verbose=0)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

plt.plot(model04.history.history['accuracy'])
plt.plot(model04.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(model04.history.history['loss'])
plt.plot(model04.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
pred=model01.predict_proba(test_X01)
for i in range(len(test_X01)):
	print("X=%s, Predicted=%s" % (test_X01[i], pred[i]))

In [ ]:
pred=model02.predict_proba(test_X02)
for i in range(len(test_X02)):
	print("X=%s, Predicted=%s" % (test_X02[i], pred[i]))

In [ ]:
pred=model03.predict_proba(test_X03)
for i in range(len(test_X03)):
	print("X=%s, Predicted=%s" % (test_X03[i], pred[i]))

In [ ]:
pred=model04.predict_proba(test_X04)
for i in range(len(test_X04)):
	print("X=%s, Predicted=%s" % (test_X04[i], pred[i]))